## Sentiment scores generation using Finbert model
 - [Finbert github](https://github.com/ProsusAI/finBERT/tree/master)
 - [Huggingface link](https://huggingface.co/ProsusAI/finbert)

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch import nn

import numpy as np
import pandas as pd

d:\Programming\sentiment-analysis-volatility-forecasting\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

d:\Programming\sentiment-analysis-volatility-forecasting\venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
def generate_sentiment_values(text: str) -> list[float]:
    inputs = tokenizer(text, return_tensors='pt', padding=True)
    logits = model(**inputs)[0]

    # Convert logits to softmax probabilities.
    probabilities = nn.functional.softmax(logits, dim=-1)

    # The probability labels are 'positive', 'negative', 'neutral' (NEED TO CONFIRM)
    return probabilities.detach().numpy().reshape((3,))

In [5]:
test_string = 'The stocks are falling'
generate_sentiment_values(test_string)

array([0.08021459, 0.26293007, 0.65685534], dtype=float32)

## Finbert on NYTimes news


1. Get sentiment for each article

In [ ]:
df = pd.read_csv('../data/tesla_gpt_summarised.csv')
#df = df[['timestamp','article_url','lead_paragraph','abstract','adjusted_date']]

In [8]:
df

,timestamp,gpt_summary
0,2021-07-26 19:46:15+00:00,"The significant increase in Tesla's profit and revenue for the second quarter, driven by a substantial increase in car sales, is likely to have a positive impact on the company both in the short and long term. This impressive financial performance can enhance investor confidence, potentially leading to a rise in stock prices as shareholders respond positively to the company's growth trajectory. Furthermore, the news could bolster Tesla's position in the competitive electric vehicle market, attracting new customers and fortifying its brand reputation as a leading innovator in the automotive industry. As a result, this strong financial report not only underscores Tesla's operational success but may also set the stage for future investments and development initiatives, further solidifying its market dominance."
1,2021-08-06 07:00:10+00:00,"The Biden administration's commitment to increasing electric vehicle (EV) sales to 50 percent of new car purchases by 2030 is poised to significantly benefit Tesla, as it already dominates the all-electric vehicle market with a robust lineup. This regulatory push creates an environment favorable for established EV manufacturers and intensifies competition for traditional automakers still reliant on internal combustion engines. As consumer demand for EVs rises, Tesla is well-positioned to capture a larger market share and strengthen its brand as a leader in sustainable transportation. Conversely, automakers resisting the transition may face declining sales and increased pressure to pivot towards electric mobility, further solidifying Tesla’s competitive advantage."
2,2021-07-22 09:00:15+00:00,"The news highlights significant operational challenges Tesla faces in its German factory, indicating a potential delay in production and showcasing a mismatch between the company's culture and local practices. This situation could negatively impact Tesla’s reputation in Europe, especially as it strives to solidify its presence in a market increasingly focused on efficiency and quality. Additionally, the ongoing production hurdles may lead to missed financial targets and affect stock performance in the near term. Nevertheless, if Tesla can adapt and eventually streamline operations, it could capitalize on Germany’s engineering prowess long-term. Overall, this development poses immediate obstacles for Tesla while also reflecting the complexities of global expansion in the automotive industry."
3,2021-08-09 17:51:08+00:00,"This news piece may have mixed implications for Tesla. On one hand, it emphasizes the growing appeal of Tesla vehicles among consumers who prioritize features and brand prestige, even at a premium price. This could reinforce Tesla's position in the luxury electric vehicle market and drive further sales among affluent buyers. On the other hand, the mention of government incentives primarily benefiting wealthier individuals could alienate average families who may feel excluded from the electric vehicle market, potentially limiting Tesla's broader customer base. Overall, while the luxury segment may remain strong for Tesla, the company's long-term growth could be challenged if it fails to expand its appeal to a more diverse audience."
4,2021-07-05 20:30:28+00:00,"The lawsuit against Tesla regarding the tragic death of a teenager in a collision involving its Model 3 while on Autopilot could significantly impact the company, particularly in terms of public perception and regulatory scrutiny. Tesla is known for its innovative approach to autonomous driving technology, but high-profile incidents, especially those resulting in loss of life, raise concerns about the safety and reliability of its systems. This legal battle could lead to increased legal liabilities, a potential decline in consumer trust, and negative media coverage, which may affect sales and stock performance. Investors might become wary of Tesla's liability exposure and future prof

In [9]:
for i, row in df.iterrows():
    output = generate_sentiment_values(row['gpt_summary'])
    df.at[i, 'pos_sentiment'] = output[0]
    df.at[i, 'neg_sentiment'] = output[1]
    df.at[i, 'neutral_sentiment'] = output[2]

    preamble = "Evaluate the above news on Tesla's stock price."
    combined_str = f"""<news>{row['gpt_summary']}</news> <instructions>{preamble}</instructions> """    
    output = generate_sentiment_values(combined_str)
    df.at[i, 'pos_sentiment_w_preamb'] = output[0]
    df.at[i, 'neg_sentiment_w_preamb'] = output[1]
    df.at[i, 'neutral_sentiment_w_preamb'] = output[2]

Save to the same file first

In [10]:
df

,timestamp,gpt_summary,pos_sentiment,neg_sentiment,neutral_sentiment,pos_sentiment_w_preamb,neg_sentiment_w_preamb,neutral_sentiment_w_preamb
0,2021-07-26 19:46:15+00:00,"The significant increase in Tesla's profit and revenue for the second quarter, driven by a substantial increase in car sales, is likely to have a positive impact on the company both in the short and long term. This impressive financial performance can enhance investor confidence, potentially leading to a rise in stock prices as shareholders respond positively to the company's growth trajectory. Furthermore, the news could bolster Tesla's position in the competitive electric vehicle market, attracting new customers and fortifying its brand reputation as a leading innovator in the automotive industry. As a result, this strong financial report not only underscores Tesla's operational success but may also set the stage for future investments and development initiatives, further solidifying its market dominance.",0.951858,0.013867,0.034274,0.952061,0.014958,0.032981
1,2021-08-06 07:00:10+00:00,"The Biden administration's commitment to increasing electric vehicle (EV) sales to 50 percent of new car purchases by 2030 is poised to significantly benefit Tesla, as it already dominates the all-electric vehicle market with a robust lineup. This regulatory push creates an environment favorable for established EV manufacturers and intensifies competition for traditional automakers still reliant on internal combustion engines. As consumer demand for EVs rises, Tesla is well-positioned to capture a larger market share and strengthen its brand as a leader in sustainable transportation. Conversely, automakers resisting the transition may face declining sales and increased pressure to pivot towards electric mobility, further solidifying Tesla’s competitive advantage.",0.934946,0.034664,0.030389,0.945068,0.026002,0.028931
2,2021-07-22 09:00:15+00:00,"The news highlights significant operational challenges Tesla faces in its German factory, indicating a potential delay in production and showcasing a mismatch between the company's culture and local practices. This situation could negatively impact Tesla’s reputation in Europe, especially as it strives to solidify its presence in a market increasingly focused on efficiency and quality. Additionally, the ongoing production hurdles may lead to missed financial targets and affect stock performance in the near term. Nevertheless, if Tesla can adapt and eventually streamline operations, it could capitalize on Germany’s engineering prowess long-term. Overall, this development poses immediate obstacles for Tesla while also reflecting the complexities of global expansion in the automotive industry.",0.048150,0.876568,0.075282,0.038152,0.897135,0.064713
3,2021-08-09 17:51:08+00:00,"This news piece may have mixed implications for Tesla. On one hand, it emphasizes the growing appeal of Tesla vehicles among consumers who prioritize features and brand prestige, even at a premium price. This could reinforce Tesla's position in the luxury electric vehicle market and drive further sales among affluent buyers. On the other hand, the mention of government incentives primarily benefiting wealthier individuals could alienate average families who may feel excluded from the electric vehicle market, potentially limiting Tesla's broader customer base. Overall, while the luxury segment may remain strong for Tesla, the company's long-term growth could be challenged if it fails to expand its appeal to a more diverse audience.",0.044077,0.899972,0.055951,0.042500,0.893700,0.063800
4,2021-07-05 20:30:28+00:00,"The lawsuit against Tesla regarding the tragic death of a teenager in a collision involving its Model 3 while on Autopilot could significantly impact the company, particularly in terms of public perception and regulatory scrutiny. Tesla is known for its innovative approach to autonomous driving technology, but high-profile incidents, especially those resu

In [ ]:
df = df.sort_values(by='timestamp')

In [ ]:
df

,timestamp,article_url,lead_paragraph,abstract,adjusted_date,gpt_summary,pos_sentiment,neg_sentiment,neutral_sentiment,pos_sentiment_w_preamb,neg_sentiment_w_preamb,neutral_sentiment_w_preamb
394,2019-10-02 10:56:22+00:00,https://www.nytimes.com/2019/10/02/business/dealbook/warren-zuckerberg-facebook.html,Good Wednesday morning. (Was this email forwarded to you? Sign up here.),"In leaked audio conversations with employees, the Facebook chief said he would take Ms. Warren to court if she tried to break up the company.",2/10/19,"While the leaked audio is primarily focused on Facebook's leadership and regulatory challenges, it is indirectly relevant to Tesla as it highlights the broader atmosphere of scrutiny facing technology companies, including social media platforms. Tesla could be impacted by this sentiment, as it operates in a similarly evolving legal and regulatory landscape with policies affecting electric vehicles and renewable energy. Such news may create increased investor wariness towards companies like Tesla, as they navigate potential government interventions or regulatory hurdles. Furthermore, it may provoke discussions about corporate accountability and market competition, influencing public perception and investor sentiment toward Tesla and its CEO, Elon Musk, who has also faced similar scrutiny in the past. Overall, the sentiment surrounding tension in large tech companies might evoke a cautious approach from investors when considering Tesla's market position.",0.027412,0.737576,0.235012,0.039174,0.625238,0.335588
50,2019-10-02 20:52:59+00:00,https://www.nytimes.com/2019/10/02/business/tesla-sales.html,"Tesla reported record quarterly deliveries on Wednesday, putting the electric-car maker within reach of its goal for the year. But analysts remain skeptical that the company will show a profit for the period.",The pace may allow the electric-car maker to fulfill its forecast for the year. But analysts are skeptical that sales are enough to yield a profit.,3/10/19,"The news of Tesla's record quarterly deliveries is a positive indicator for the company, suggesting strong demand and execution in the competitive electric vehicle market. Achieving its delivery goals reinforces Tesla's growth potential and may bolster investor confidence; however, the lingering skepticism from analysts regarding profitability for the quarter highlights a critical concern that could temper optimism. If Tesla continues to struggle with profit margins despite high sales volumes, investor sentiment might shift, potentially impacting stock performance and future investment. Overall, while the record deliveries serve as a strong achievement, the uncertainty around profitability introduces a cautious note that could curtail positive sentiment in the short term.",0.100441,0.863928,0.035632,0.291604,0.663051,0.045344
207,2019-10-09 10:12:51+00:00,https://www.nytimes.com/2019/10/09/business/dealbook/china-companies-nba.html,Good Wednesdayg. Breaking: The O.E.C.D. proposed new corporate tax rules this morning that would make it harder for multinationals to minimize their tax bills by shifting profits to different regions. (Sign up here to get this in your inbox every morning.),"It once was clear what American businesses needed to avoid to succeed in China’s huge market. As the N.B.A. has shown, that’s no longer the case.",9/10/19,"The OECD's proposed new corporate tax rules may significantly impact Tesla by limiting its ability to minimize tax liabilities through profit shifting across regions, which could increase overall tax obligations for the company. Given Tesla's multinational operations, particularly in key markets like China, any added financial burden from corporate taxes could affect its pricing strategies, profit margins, and ultimately its competitiveness in the global electric vehicle market. Additionally, as Tesla navigates the complexities of doing business in China, these regulations might further complicate its strategic and oper

In [ ]:
df.to_csv('../data/tesla_headlines_gpt_summarised.csv', index=False)

2. Group by date

In [36]:
new_df = pd.read_csv('../data/tesla_headlines_gpt_summarised.csv')

In [37]:
new_df

,timestamp,article_url,lead_paragraph,abstract,adjusted_date,gpt_summary,pos_sentiment,neg_sentiment,neutral_sentiment,pos_sentiment_w_preamb,neg_sentiment_w_preamb,neutral_sentiment_w_preamb
0,2019-10-02 10:56:22+00:00,https://www.nytimes.com/2019/10/02/business/dealbook/warren-zuckerberg-facebook.html,Good Wednesday morning. (Was this email forwarded to you? Sign up here.),"In leaked audio conversations with employees, the Facebook chief said he would take Ms. Warren to court if she tried to break up the company.",2/10/19,"While the leaked audio is primarily focused on Facebook's leadership and regulatory challenges, it is indirectly relevant to Tesla as it highlights the broader atmosphere of scrutiny facing technology companies, including social media platforms. Tesla could be impacted by this sentiment, as it operates in a similarly evolving legal and regulatory landscape with policies affecting electric vehicles and renewable energy. Such news may create increased investor wariness towards companies like Tesla, as they navigate potential government interventions or regulatory hurdles. Furthermore, it may provoke discussions about corporate accountability and market competition, influencing public perception and investor sentiment toward Tesla and its CEO, Elon Musk, who has also faced similar scrutiny in the past. Overall, the sentiment surrounding tension in large tech companies might evoke a cautious approach from investors when considering Tesla's market position.",0.027412,0.737576,0.235012,0.039174,0.625238,0.335588
1,2019-10-02 20:52:59+00:00,https://www.nytimes.com/2019/10/02/business/tesla-sales.html,"Tesla reported record quarterly deliveries on Wednesday, putting the electric-car maker within reach of its goal for the year. But analysts remain skeptical that the company will show a profit for the period.",The pace may allow the electric-car maker to fulfill its forecast for the year. But analysts are skeptical that sales are enough to yield a profit.,3/10/19,"The news of Tesla's record quarterly deliveries is a positive indicator for the company, suggesting strong demand and execution in the competitive electric vehicle market. Achieving its delivery goals reinforces Tesla's growth potential and may bolster investor confidence; however, the lingering skepticism from analysts regarding profitability for the quarter highlights a critical concern that could temper optimism. If Tesla continues to struggle with profit margins despite high sales volumes, investor sentiment might shift, potentially impacting stock performance and future investment. Overall, while the record deliveries serve as a strong achievement, the uncertainty around profitability introduces a cautious note that could curtail positive sentiment in the short term.",0.100441,0.863928,0.035632,0.291604,0.663051,0.045344
2,2019-10-09 10:12:51+00:00,https://www.nytimes.com/2019/10/09/business/dealbook/china-companies-nba.html,Good Wednesdayg. Breaking: The O.E.C.D. proposed new corporate tax rules this morning that would make it harder for multinationals to minimize their tax bills by shifting profits to different regions. (Sign up here to get this in your inbox every morning.),"It once was clear what American businesses needed to avoid to succeed in China’s huge market. As the N.B.A. has shown, that’s no longer the case.",9/10/19,"The OECD's proposed new corporate tax rules may significantly impact Tesla by limiting its ability to minimize tax liabilities through profit shifting across regions, which could increase overall tax obligations for the company. Given Tesla's multinational operations, particularly in key markets like China, any added financial burden from corporate taxes could affect its pricing strategies, profit margins, and ultimately its competitiveness in the global electric vehicle market. Additionally, as Tesla navigates the complexities of doing business in China, these regulations might further complicate its strategic and operation

In [33]:
agg_func = {
    'pos_sentiment': 'mean',
    'neg_sentiment': 'mean',
    'neutral_sentiment': 'mean',
    'pos_sentiment_w_preamb': 'mean',
    'neg_sentiment_w_preamb': 'mean',
    'neutral_sentiment_w_preamb': 'mean'
}
column_rename = {
    'pos_sentiment': 'mean_pos_sentiment',
    'neg_sentiment': 'mean_neg_sentiment',
    'neutral_sentiment': 'mean_neutral_sentiment',
    'pos_sentiment_w_preamb': 'mean_pos_preamble_sentiment',
    'neg_sentiment_w_preamb': 'mean_neg_preamble_sentiment',
    'neutral_sentiment_w_preamb': 'mean_neutral_preamble_sentiment'
}
grouped_by_date_df = new_df.groupby(by='adjusted_date').agg(agg_func).rename(columns=column_rename).reset_index()
grouped_by_date_df

,adjusted_date,mean_pos_sentiment,mean_neg_sentiment,mean_neutral_sentiment,mean_pos_preamble_sentiment,mean_neg_preamble_sentiment,mean_neutral_preamble_sentiment
0,1/1/24,0.257112,0.560906,0.181982,0.481200,0.408570,0.110230
1,1/10/21,0.144524,0.786026,0.069450,0.154926,0.794357,0.050718
2,1/11/23,0.334139,0.621210,0.044651,0.353742,0.596856,0.049402
3,1/12/20,0.450078,0.220043,0.329879,0.695714,0.094795,0.209491
4,1/12/23,0.059800,0.904017,0.036183,0.159795,0.797815,0.042389
...,...,...,...,...,...,...,...
857,9/8/22,0.026218,0.887631,0.086151,0.029794,0.879863,0.090342
858,9/9/20,0.075703,0.606183,0.318115,0.077259,0.710299,0.212441
859,9/9/21,0.285432,0.379544,0.335024,0.145217,0.586420,0.268363
860,9/9/22,0.826727,0.018678,0.154595,0.913103,0.012185,0.074712


In [35]:
grouped_by_date_df.to_csv('../data/tesla_sentiment_gpt_summarised.csv', index=False)

## Merge the Sentiments DataFrame into 1 

In [48]:
nyt_sentiment = pd.read_csv('../data/nyt_sentiment.csv')
tesla_sentiment = pd.read_csv('../data/tesla_sentiment.csv')


In [50]:
nyt_sentiment['News'] = 'Market News'

In [61]:
nyt_sentiment

,adjusted_date,mean_pos_sentiment,mean_neg_sentiment,mean_neutral_sentiment,mean_pos_preamble_sentiment,mean_neg_preamble_sentiment,mean_neutral_preamble_sentiment,News
0,2019-09-30,0.056308,0.762604,0.181088,0.127192,0.641141,0.231667,Market News
1,2019-10-01,0.084752,0.525347,0.389900,0.069554,0.531837,0.398609,Market News
2,2019-10-02,0.064854,0.476130,0.459017,0.053761,0.438076,0.508164,Market News
3,2019-10-03,0.267430,0.271210,0.461360,0.250733,0.239351,0.509915,Market News
4,2019-10-04,0.132889,0.533843,0.333267,0.070609,0.499317,0.430075,Market News
...,...,...,...,...,...,...,...,...
1493,2024-09-25,0.159386,0.285289,0.555325,0.112141,0.281216,0.606643,Market News
1494,2024-09-26,0.296828,0.350619,0.352554,0.260930,0.321001,0.418069,Market News
1495,2024-09-27,0.192744,0.503622,0.303634,0.209407,0.408375,0.382218,Market News
1496,2024-09-28,0.225603,0.226435,0.547962,0.209130,0.203006,0.587863,Market News


In [62]:
nyt_sentiment = nyt_sentiment[:-1]

In [51]:
tesla_sentiment['News'] = 'Tesla'

In [63]:
tesla_sentiment

,adjusted_date,mean_pos_sentiment,mean_neg_sentiment,mean_neutral_sentiment,mean_pos_preamble_sentiment,mean_neg_preamble_sentiment,mean_neutral_preamble_sentiment,News
0,1/1/24,0.932750,0.018450,0.048801,0.883637,0.019422,0.096940,Tesla
1,1/10/21,0.007312,0.974635,0.018053,0.007583,0.970156,0.022261,Tesla
2,1/11/23,0.260249,0.570400,0.169351,0.224915,0.534778,0.240307,Tesla
3,1/12/20,0.068412,0.030966,0.900621,0.037842,0.027356,0.934802,Tesla
4,1/12/23,0.008780,0.971812,0.019408,0.012819,0.961352,0.025829,Tesla
...,...,...,...,...,...,...,...,...
857,9/8/22,0.061572,0.221923,0.716505,0.070154,0.102857,0.826989,Tesla
858,9/9/20,0.064603,0.097978,0.837419,0.056757,0.021830,0.921414,Tesla
859,9/9/21,0.075175,0.014510,0.910315,0.087085,0.011177,0.901738,Tesla
860,9/9/22,0.763597,0.009088,0.227315,0.504756,0.011038,0.484206,Tesla


In [65]:
tesla_sentiment['adjusted_date'] = pd.to_datetime(tesla_sentiment['adjusted_date'], format="%d/%m/%y").dt.strftime("%Y-%m-%d")


In [67]:
union_df = pd.concat([tesla_sentiment, nyt_sentiment], ignore_index=True)